cài đặt pip

In [ ]:
!pip install flask-ngrok
!pip install flask-CORS
!pip install pyngrok
!pip freeze | grep flask_cors

open drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!ls '/content/drive/MyDrive/ColabNotebooks/DF_test/'

**conncet ngrok**

In [ ]:
!ngrok authtoken "your-token"

In [ ]:
from flask import Flask, request, render_template
from pyngrok import ngrok
app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import numpy as np
import cv2
import os
import tensorflow as tf
from pyngrok import ngrok
import threading

port = 5000
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url
# ... Update inbound traffic via APIs to use the public-facing ngrok URL

# Khởi tạo Flask app
app = Flask(__name__, template_folder='/content/drive/MyDrive/ColabNotebooks/DF_test/', static_folder='/content/drive/MyDrive/ColabNotebooks/DF_test/assets', static_url_path='/assets')
CORS(app)

# Load model
model = tf.keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/DF_test/full_modeldf.h5')

# Load face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Tạo thư mục temp
if not os.path.exists('temp'):
    os.makedirs('temp')

def preprocess_image(image):
    image = cv2.resize(image, (128, 128))
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Route cho trang chủ
@app.route('/')
def home():
    return render_template('index.html')
@app.route('/test')
def test():
   return render_template('test.html')
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        try:
            # Lưu file tạm thời
            file_path = os.path.join('temp', file.filename)
            file.save(file_path)

            # Đọc và xử lý ảnh
            img = cv2.imread(file_path)
            if img is None:
                return jsonify({'error': 'Invalid image file'}), 400

            # Kiểm tra khuôn mặt
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            if len(faces) == 0:
                os.remove(file_path)
                return jsonify({'result': 'Not a human face'}), 200

            # Xử lý ảnh cho model
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            processed_image = preprocess_image(img)

            # Dự đoán
            prediction = model.predict(processed_image)
            pred_label = np.argmax(prediction)

            # Xóa file tạm
            os.remove(file_path)

            # Xử lý kết quả
            result = 'Real' if pred_label == 1 else 'Fake'
            confidence = float(prediction[0][pred_label])

            return jsonify({
                'result': result,
                'confidence': round(confidence * 100, 2)
            }), 200

        except Exception as e:
            if os.path.exists(file_path):
                os.remove(file_path)
            return jsonify({'error': str(e)}), 500

threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()